In [1]:
import requests
import gzip
import shutil

In [2]:
def extract_data(url, file_path):
    '''
    method to extract data
    '''
    with requests.session() as c:
        r = requests.get(url, allow_redirects=True)
        open(file_path, 'wb').write(r.content)

    with gzip.open(file_path, 'rb') as f_in:
        with open(file_path[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
# urls
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
rating_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

extract_data(basics_url,'title.basics.tsv.gz')
extract_data(rating_url,'title.ratings.tsv.gz')

In [5]:
import pandas as pd
import os

basics_data_path = os.path.join(os.getcwd(), 'title.basics.tsv')
ratings_data_path = os.path.join(os.getcwd(), 'title.ratings.tsv')
basicsDf = pd.read_csv(basics_data_path, sep='\t')
  
ratingsDf = pd.read_csv(ratings_data_path, sep='\t')
new_df = pd.merge(ratingsDf, basicsDf, on='tconst')

c:\python35\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
sortedRatingsDf = new_df.sort_values(by=['averageRating'],ascending=False)
# Only select the movies with more than 50000 votes
withLargeVotesDf = sortedRatingsDf.loc[(sortedRatingsDf['numVotes'] > 50000) & (sortedRatingsDf['titleType'].isin(['movie']))]
print(withLargeVotesDf.head())

           tconst  averageRating  numVotes titleType  \
759431  tt5813916            9.6     94914     movie   
78394   tt0111161            9.3   1927527     movie   
42947   tt0068646            9.2   1318147     movie   
45420   tt0071562            9.0    910562     movie   
232197  tt0468569            9.0   1898605     movie   

                    primaryTitle             originalTitle  isAdult startYear  \
759431           The Mountain II                    Dag II        0      2016   
78394   The Shawshank Redemption  The Shawshank Redemption        0      1994   
42947              The Godfather             The Godfather        0      1972   
45420     The Godfather: Part II    The Godfather: Part II        0      1974   
232197           The Dark Knight           The Dark Knight        0      2008   

       endYear runtimeMinutes                 genres  
759431      \N            135              Drama,War  
78394       \N            142            Crime,Drama  
42947      

In [14]:
writer = pd.ExcelWriter('Top500Movies.xlsx')
withLargeVotesDf.head(500).to_excel(writer,'Sheet1')
writer.save()